In [1]:
from rdkit import Chem
import numpy as np
import pandas as pd

#If one SMILES consists of seperated anion part and cation part, only the part containing P atom is remained.

class Molecule:
    def __init__(self, smiles):
        self.smiles = smiles
        self.mol = Chem.MolFromSmiles(smiles)
        if self.mol == None:
            print(smiles)
        self.mol_H = Chem.AddHs(self.mol)
        self.P_id = self.get_P_index()
        self.P_neighbor = self.get_P_neighbor_idx()
        
    def only_one_P(self):
        '''Check if there is only one P atom in the molecule'''
        count = 0
        for atom in self.mol.GetAtoms():
            atom_symbol = atom.GetSymbol()
            if atom_symbol == "P":count+=1
        if count == 0:
            print(self.smiles)
            raise ValueError("No P atom in the molecule, please check it")
        elif count == 1:
            return True
        else:
            return False
        
    def get_P_index(self):
        '''Get the index of P atom in the molecule. Record the first atom if multiple P atoms exist'''
        for atom in self.mol.GetAtoms():
            if atom.GetSymbol() == "P":
                self.P_id = atom.GetIdx()
                return  self.P_id
            
    def get_P_neighbor_idx(self):
        '''Find the neighbor atoms of the P atom from get_P_index'''
        if self.P_id is None:
            self.P_id = self.get_P_index()
        self.P_neighbor_idx = [x.GetIdx() for x in self.mol.GetAtomWithIdx(self.P_id).GetNeighbors()]
        
        return self.P_neighbor_idx
    
    def is_all_C_neighbor(self):
        '''Check if there is only C atoms in the neighbor atoms of the P atom'''
        if self.P_neighbor == None:
            self.P_neighbor = self.get_P_neighbor_idx()
        
        P_neighbor_H_idx = [x.GetIdx() for x in self.mol_H.GetAtomWithIdx(self.P_id).GetNeighbors()]
        for neighbor in P_neighbor_H_idx:
            if self.mol_H.GetAtomWithIdx(neighbor).GetSymbol() != "C":return False
        return True
        
    def S_in_P_neighbor(self):
        '''Check if there is any S atom in the neighbor atoms of the P atom'''
        if self.P_neighbor is None:
            self.P_neighbor = self.get_P_neighbor_idx()
        for idx in self.P_neighbor:
            if self.mol.GetAtomWithIdx(idx).GetSymbol() == "S":
                return True
        return False
    
    def ONHBr_in_P_neighbor(self):
        '''Check if there is any O or H orBr or H atom in the neighbor atoms of the P atom'''
        if self.P_neighbor is None:
            self.P_neighbor = self.get_P_neighbor_idx()
        P_neighbor_H_idx = [x.GetIdx() for x in self.mol_H.GetAtomWithIdx(self.P_id).GetNeighbors()]
        
        for idx in P_neighbor_H_idx:
            symbol = self.mol_H.GetAtomWithIdx(idx).GetSymbol()
            if symbol == "O" or symbol == 'N' or symbol == 'Br' or symbol == "H":
                return True
        return False
    
    def Cl_in_P_neighbor(self):
        '''Check if there is any Cl atom in the neighbor atoms of P atom'''
        if self.P_neighbor is None:
            self.P_neighbor = self.get_P_neighbor_idx()
    
        for idx in self.P_neighbor:
            symbol = self.mol.GetAtomWithIdx(idx).GetSymbol()
            if symbol == "Cl":
                return True
        return False
    
    def S_in_mol(self):
        '''Check if there is any S atom in the molecule'''
        for atom in self.mol.GetAtoms():
            if atom.GetSymbol() == "S":
                return True
        return False
    
    def only_PCH(self):
        '''Check if only C and H and P atoms exist in the molecule'''
        for atom in self.mol.GetAtoms():
            if atom.GetSymbol() != "C" and atom.GetSymbol() != "H" and atom.GetSymbol() != "P" :
                return False
        return True 
        
        
    def is_contain_scycle(self):
        '''Check if there is any ring with less than 6 members connecting to P. P is never in a ring in our dataset
        !!! Adjusted for featurization !!!
        ''' 
        try:
            if not self.only_PCH:raise ValueError("The molecule isn't only PCH for cycle judging")
            rings = self.mol.GetRingInfo().AtomRings()
            if len(rings) == 0:return False
            for ring in rings:
                if len(ring) < 6:return True        
            return False
        except:
            return False
        
    @staticmethod
    def remain_neighbor(frag_mol,neighbor_ids,ring_ids,all_neighbor_ids):
        '''Check if there is any atom not be visited '''
        is_new_neighbor = False
        for neighbor_id in neighbor_ids:
            new_ids = [x.GetIdx() for x in frag_mol.GetAtomWithIdx(neighbor_id).GetNeighbors()]
            for new_id in new_ids:
                if not new_id in ring_ids and not new_id in all_neighbor_ids:
                    is_new_neighbor = True
        return is_new_neighbor
                    

    def max_chain_distance(self,imsmiles):
        '''Split the molecule into fragments connecting to P, than apply a traversal examination process to calculate length of each chain. Finally return the max chain length from P (Unit: 1 in graph distance matrix).'''
        #let fragment start from the linked atom
        if imsmiles[0] == "*":
            imsmiles = imsmiles[1:]
        else:
            star_id = imsmiles.find("*")
            imsmiles = imsmiles[star_id+2:]
        frag_mol = Chem.MolFromSmiles(imsmiles)
        initial_id = 0
        rings = frag_mol.GetRingInfo().AtomRings()
        ring_ids = []
        for ring in rings:
            ring_ids.extend(list(ring))
        if initial_id in ring_ids:return 100
        else:
            distance_mtx = Chem.rdmolops.GetDistanceMatrix(frag_mol)[0]
            all_neighbor_ids = []
            new_neighbor = [initial_id]
            visited_ids = []
            all_neighbor_ids.append(initial_id)
            while self.remain_neighbor(frag_mol,new_neighbor,ring_ids,all_neighbor_ids):
                instance_neighbor = []
                for neighbor in new_neighbor:
                    if not neighbor in visited_ids:
                        visited_ids.append(neighbor)
                        to_add_neighbors = [x.GetIdx() for x in frag_mol.GetAtomWithIdx(neighbor).GetNeighbors()]
                        if len(to_add_neighbors) == 0:continue
                        else:
                            for to_add_neighbor in to_add_neighbors:
                                if not to_add_neighbor in ring_ids and not to_add_neighbor in all_neighbor_ids:
                                    instance_neighbor.append(to_add_neighbor)
                            
                all_neighbor_ids.extend(instance_neighbor)
                new_neighbor = instance_neighbor
            distance = [distance_mtx[id] for id in all_neighbor_ids]
            max_distance = np.max(distance)

        return max_distance

    def is_connected_chain(self):
        '''check if there is any carbon chain with a length less than 4 connected to P
        !!! Adjusted for featurization !!!
        '''
        if not self.only_PCH:raise ValueError("The molecule isn't only PCH for chain judging")
        try:
            if self.P_id is None:
                self.P_id = self.get_P_index()
            if self.P_neighbor is None:
                self.P_neighbor = self.get_P_neighbor_idx()
            bonds_list = [self.mol.GetBondBetweenAtoms(neighbor, self.P_id).GetIdx() for neighbor in self.P_neighbor_idx]
            im_res = Chem.FragmentOnBonds(self.mol, bonds_list)
            im_frag = Chem.MolToSmiles(im_res)
            imsmiles_lst = im_frag.split(".")
            for imsmiles in imsmiles_lst:
                if not "P" in imsmiles:
                    frag_length = self.max_chain_distance(imsmiles) + 1
                    if 0 < frag_length < 4:return True
            return False
        except:
            return False


In [2]:
P_data = pd.read_csv('./dataset.csv')
P_data['target_cla'] = 0
for i in range(len(P_data)):
    if P_data.loc[i,'effect(delta C2+)'] >= 0.05:
        P_data.loc[i,'target_cla'] = 1

In [3]:
P_data['target_cla'].value_counts()

target_cla
1    65
0    62
Name: count, dtype: int64

In [8]:
P_data['target_cla'].to_csv("target_cla.csv",index=None)

In [6]:
P_data['is_all_C_neighbor'] = None
P_data['S_in_P_neighbor'] = None
# P_data['ONHBr_in_P_neighbor'] = None
P_data['Cl_in_P_neighbor'] = None
P_data['S_in_mol'] = None
P_data['only_PCH'] = None
P_data['is_contain_scycle'] = None
P_data['is_connected_chain'] = None



for i in range(len(P_data)):
    smiles = P_data.iloc[i,1].strip()
    # effect = P_data.iloc[i,2]
    m = Molecule(smiles)
    match m.is_all_C_neighbor():
        case True:
            P_data.loc[i,'is_all_C_neighbor'] = 1
        case False:
            P_data.loc[i,'is_all_C_neighbor'] = 0
        case _:pass
    match m.S_in_P_neighbor():
        case True:
            P_data.loc[i,'S_in_P_neighbor'] = 1
        case False:
            P_data.loc[i,'S_in_P_neighbor'] = 0
        case _:pass
    # match m.ONHBr_in_P_neighbor():
    #     case True:
    #         P_data.loc[i,'ONHBr_in_P_neighbor'] = 1
    #     case False:
    #         P_data.loc[i,'ONHBr_in_P_neighbor'] = 0
        # case _:pass
    match m.Cl_in_P_neighbor():
        case True:
            P_data.loc[i,'Cl_in_P_neighbor'] = 1
        case False:
            P_data.loc[i,'Cl_in_P_neighbor'] = 0
        case _:pass
    match m.S_in_mol():
        case True:
            P_data.loc[i,'S_in_mol'] = 1
        case False:
            P_data.loc[i,'S_in_mol'] = 0
        case _:pass
    match m.only_PCH():
        case True:
            P_data.loc[i,'only_PCH'] = 1
        case False:
            P_data.loc[i,'only_PCH'] = 0
        case _:pass
    match m.is_contain_scycle():
        case True:
            P_data.loc[i,'is_contain_scycle'] = 1
        case False:
            P_data.loc[i,'is_contain_scycle'] = 0
        case _:pass
    match m.is_connected_chain():
        case True:
            P_data.loc[i,'is_connected_chain'] = 1
        case False:
            P_data.loc[i,'is_connected_chain'] = 0
        case _:pass
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65


[20:58:10] SMILES Parse Error: syntax error while parsing: =S
[20:58:10] SMILES Parse Error: Failed parsing SMILES '=S' for input: '=S'
[20:58:10] SMILES Parse Error: syntax error while parsing: =O
[20:58:10] SMILES Parse Error: Failed parsing SMILES '=O' for input: '=O'
[20:58:10] SMILES Parse Error: syntax error while parsing: =O
[20:58:10] SMILES Parse Error: Failed parsing SMILES '=O' for input: '=O'
[20:58:10] SMILES Parse Error: syntax error while parsing: =O
[20:58:10] SMILES Parse Error: Failed parsing SMILES '=O' for input: '=O'
[20:58:10] SMILES Parse Error: syntax error while parsing: =O
[20:58:10] SMILES Parse Error: Failed parsing SMILES '=O' for input: '=O'
[20:58:11] non-ring atom 0 marked aromatic
[20:58:11] SMILES Parse Error: syntax error while parsing: =S
[20:58:11] SMILES Parse Error: Failed parsing SMILES '=S' for input: '=S'
[20:58:11] SMILES Parse Error: syntax error while parsing: =O
[20:58:11] SMILES Parse Error: Failed parsing SMILES '=O' for input: '=O'
[20:5

66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126


# Train models

In [13]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn import tree
import matplotlib.pyplot as plt
import dtreeviz
import os
from matplotlib.font_manager import FontProperties

font = FontProperties(fname="../ARIAL.TTF", size=11)

DIR = "Decision_Trees/Human0103"
os.makedirs(DIR,exist_ok=True)
# Load the features and target
features = pd.read_csv("tree_features0103.csv")
target = pd.read_csv("target_cla.csv")

feature_names = list(features.columns)
# Convert target to a 1D numpy array (if necessary)
target = target.values.ravel()

# Initialize the Decision Tree Classifier
dt_clf = DecisionTreeClassifier(max_depth=5)

# Initialize KFold cross-validator
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Arrays to store predictions for the train and test sets
train_predictions = np.zeros(len(features))
test_predictions = np.zeros(len(features))

# Perform 5-Fold Cross-Validation
fold = 1  # To track the current fold
for train_index, test_index in kf.split(features):
    # Split the data into train and test sets
    X_train, X_test = features.iloc[train_index], features.iloc[test_index]
    y_train, y_test = target[train_index], target[test_index]
    
    # Train the Decision Tree Classifier
    dt_clf.fit(X_train, y_train)
    
    # Display the structure of the current Decision Tree
    print(f"\nDecision Tree Structure for Fold {fold}:\n")
    # tree_structure = export_text(dt_clf, feature_names=list(features.columns))
    # print(tree_structure)

    #plot_tree
    # plt.figure(facecolor= 'g')
    # a = tree.plot_tree(dt_clf, feature_names=list(features.columns),rounded=True,filled=True)
    
    viz_model = dtreeviz.model(dt_clf, 
                X_train=X_train,
                y_train=y_train,
                feature_names=list(features.columns), 
                class_names=['Low FE','High FE'],
                # title="Decision Tree - Iris data set"
                )
    #plt.show()
    plot = viz_model.view()
    plot.save(f'{DIR}/fold_{fold}.svg')
    fold += 1

    # Predict on the train set (using train indices)
    train_pred = dt_clf.predict(X_train)
    train_predictions[train_index] = train_pred  # Collect train predictions for these indices
    
    # Predict on the test set (using test indices)
    test_pred = dt_clf.predict(X_test)
    test_predictions[test_index] = test_pred  # Collect test predictions for these indices

# Calculate final train and test accuracy
train_accuracy = accuracy_score(target, train_predictions)
test_accuracy = accuracy_score(target, test_predictions)

# Print the results
print(f"\nFinal Train Accuracy: {train_accuracy:.4f}")
print(f"Final Test Accuracy: {test_accuracy:.4f}")


Decision Tree Structure for Fold 1:



/home/lnh/anaconda3/envs/data/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names



Decision Tree Structure for Fold 2:



/home/lnh/anaconda3/envs/data/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names



Decision Tree Structure for Fold 3:



...


Decision Tree Structure for Fold 4:



/home/lnh/anaconda3/envs/data/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names



Decision Tree Structure for Fold 5:



/home/lnh/anaconda3/envs/data/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names



Final Train Accuracy: 0.8189
Final Test Accuracy: 0.7874


In [19]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import os

DIR = "Decision_Trees/MFF"
os.makedirs(DIR,exist_ok=True)
# Load the features and target
features = pd.read_csv("../MFF/MFF_features_0.02.csv")
target = pd.read_csv("target_cla.csv")

# Convert target to a 1D numpy array (if necessary)
target = target.values.ravel()

# Initialize the Random Forest Classifier
dt_clf = DecisionTreeClassifier(max_depth=5)

# Initialize KFold cross-validator
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Arrays to store predictions for the train and test sets
train_predictions = np.zeros(len(features))
test_predictions = np.zeros(len(features))

fold = 1  # To track the current fold
# Perform 5-Fold Cross-Validation
for train_index, test_index in kf.split(features):
    # Split the data into train and test sets
    X_train, X_test = features.iloc[train_index], features.iloc[test_index]
    y_train, y_test = target[train_index], target[test_index]
    
    # Train the Random Forest Classifier
    dt_clf.fit(X_train, y_train)
    # print(f"\nDecision Tree Structure for Fold {fold}:\n")
    # tree_structure = export_text(rf_clf, feature_names=list(features.columns))
    # print(tree_structure)
    viz_model = dtreeviz.model(dt_clf, 
                X_train=X_train,
                y_train=y_train,
                feature_names=list(features.columns), 
                class_names=['Low FE','High FE'],
                # title="Decision Tree - Iris data set"
                )
    #plt.show()
    plot = viz_model.view()
    plot.save(f'{DIR}/fold_{fold}.svg')
    fold += 1
    # Predict on the train set (using train indices)
    train_pred = rf_clf.predict(X_train)
    train_predictions[train_index] = train_pred  # Collect train predictions for these indices
    
    # Predict on the test set (using test indices)
    test_pred = rf_clf.predict(X_test)
    test_predictions[test_index] = test_pred  # Collect test predictions for these indices

# Calculate final train and test accuracy
train_accuracy = accuracy_score(target, train_predictions)
test_accuracy = accuracy_score(target, test_predictions)

# Print the results
print(f"Final Train Accuracy: {train_accuracy:.4f}")
print(f"Final Test Accuracy: {test_accuracy:.4f}")

/home/lnh/anaconda3/envs/data/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
/home/lnh/anaconda3/envs/data/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
/home/lnh/anaconda3/envs/data/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
/home/lnh/anaconda3/envs/data/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
/home/lnh/anaconda3/envs/data/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names


Final Train Accuracy: 0.8425
Final Test Accuracy: 0.8425


In [ ]:
rf_clf = DecisionTreeClassifier(max_depth=5)
kf = KFold(n_splits=5, shuffle=True, random_state=42)
# Accuracy of Tree features
Final Train Accuracy: 0.8189
Final Test Accuracy: 0.7795
# Accuracy of MFF features
Final Train Accuracy: 0.8425
Final Test Accuracy: 0.7402